In [ ]:
%matplotlib notebook
import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
# athenacli -e prod -w3 -n0 -p KET -o . EEGTEST

### Load the montage of sensor channel locations and set up the files to process.

In [ ]:
montage = mne.channels.read_montage('standard_1020')
#edf_file = 'EEG_TEST_0001_raw.edf'
#log_file = 'log3.csv'
edf_file = os.path.expanduser('~/data/eeg/20190701/aditya_TEST_raw.edf')
log_file = os.path.expanduser('~/data/eeg/20190701/adityaTest.csv')

In [ ]:
raw = mne.io.read_raw_edf(edf_file, stim_channel='Trigger', eog=['EEG X1-Pz'], 
                          misc=['EEG CM-Pz','EEG X2-Pz','EEG X3-Pz'])
# Rename the channels so they match the standard montage channel names
raw.rename_channels({c:c.replace('EEG ','').replace('-Pz','') for c in raw.ch_names})
raw.set_montage(montage)
eeg_sample_interval_ms = 1/raw.info['sfreq'] * 1000
print(raw.info)
#raw.plot_sensors()

### Find the events

In [ ]:
events = mne.find_events(raw)

In [ ]:
logdf = pd.read_csv(log_file, header=None, names=['client_ts','trigger_ts','rtdelay','msg','uid'])
logdf.client_ts = (logdf.client_ts * 1000).round().astype(int)
logdf.trigger_ts = (logdf.trigger_ts * 1000).round().astype(int)
logdf['bytecode'] = logdf.client_ts % 255 + 1
logdf = logdf.sort_values('client_ts').reset_index(drop=True)
#phone_start = 722
#logdf = logdf.iloc[722:, :]
logdf.head()

In [ ]:
eventdf = pd.DataFrame(events, columns=['time_idx','prev_diff','bytecode'])
# NOTE: the eeg timestamp is local time, not UTC! Be sure to use the correct adjustment here.
start_ts = int(raw.info['meas_date'][0]) + 7*60*60
eventdf['eeg_ts'] = ((eventdf.time_idx / 300 + start_ts) * 1000).round().astype(int)
eventdf.head()

In [ ]:
window = 30000 # +/-, in milliseconds
matching_indices = []
for event_idx in range(eventdf.shape[0] - 2):
    tmp = logdf.loc[np.abs(eventdf.eeg_ts[event_idx] - logdf.trigger_ts) < window, :]
    matches = np.argwhere(tmp.bytecode == eventdf.bytecode[event_idx])
    if len(matches) > 0:
        for match in matches:
            # See if the next two match. If so, add this to the list
            try:
                if (tmp.bytecode[match[0]+1] == eventdf.bytecode[event_idx+1] 
                    and tmp.bytecode[match[0]+1] == eventdf.bytecode[event_idx+1]):
                    matching_indices.append((event_idx,match[0]))
                    continue
            except:
                pass

        
print(matching_indices)
# WORK HERE
# THIS ISN"T WORKING-- NEEDS DEBUGGING


### Find the optimal fuzzy alignment between the log file and event bytecode sequence

In [ ]:
# Assume clocks are roughly matched (within a few seconds)
logn = logdf.shape[0]
eventn = eventdf.shape[0]
deltas = np.abs(logdf.trigger_ts - eventdf.loc[eventdf_idx,'eeg_ts'])
logdf_idx = deltas.idxmin()
eventdf.head()

In [ ]:
matching_pts = np.array([[1562017755197, 1562017748210], [1562018037437,1562018040407], [1562019562090,1562019570690]])
y = matching_pts[:,0]
x = matching_pts[:,1]
offset = x[0]
x = x - offset
y = y - offset
p = np.polyfit(x, y, 1)
plt.plot(x, y, 'ro-', x, np.polyval(p,x), 'b-')

### Find the best-matching bytecode for each event

In [ ]:
# Compute the time delta to 
target_bytecode = eventdf.bytecode[0]
target_time_idx = eventdf.time_idx[0]
logdf['delta_to_event_start'] = logdf.trigger_ts - logdf.trigger_ts[logdf_idx]
#logdf.loc[logdf_idx-10:logdf_idx+10]
logdf.head()

In [ ]:
window = 30000 # +/-, in milliseconds
for event_idx in range(eventdf.shape[0]):
    tmp = logdf.loc[np.abs(eventdf.eeg_ts - logdf.trigger_ts) < window, :]
    
    
err = (logdf.loc[logdf.delta_to_event_center.abs() < window, 'bytecode'] - target_bytecode).abs()
logdf_idx = err.idxmin()
if err[logdf_idx] == 0:
    print('Found an exactly matching bytecode (%d)!' % target_bytecode)
else:
    print('Closes matching bytecode for %d is %d.' %(target_bytecode, logdf.bytecode[logdf_idx]))

# Compute event clock bias
event_to_client_bias = eventdf.eeg_ts[eventdf_idx] - logdf.client_ts[logdf_idx]
#netdelay = logdf.rtdelay[logdf.rtdelay>0].min()
netdelay = logdf.rtdelay[logdf_idx] / 2
if netdelay<=0:
    netdelay = logdf.rtdelay[logdf.rtdelay>0].median() / 2
# The bias is how far ahead (positive bias) or behind (negative) the EEG clock is relative to the client clock. 
# Since the event arrive netdelay milliseconds late, subtract netdelay from this bias. E.g., if we compute the bias
# as 1000 ms, it should actually be 900 for a 100ms netdelay, since packets arrive 100ms after being sent.
corrected_bias = event_to_client_bias - netdelay
print('EEG clock bias is %d ms; netdelay = %d ms; corrected_bias = %d' % (event_to_client_bias, netdelay, corrected_bias))

logdf['event_time_ms'] = (logdf.client_ts - logdf.client_ts[logdf_idx] + corrected_bias)
logdf['event_time_idx'] = (logdf.event_time_ms / (1000./300.) + target_time_idx).round().astype(int)

In [ ]:
eventdf.eeg_ts[eventdf_idx] - logdf.client_ts[logdf_idx]

In [ ]:
#logdf.loc[logdf_idx-10:logdf_idx+10]
logdf.loc[0:,:].head(40)

In [ ]:
#np.polyfit(logdf.eeg_event_ts, logdf.client_ts/1000, 1)
plt.plot(logdf.eeg_event_ts, logdf.client_ts/1000)

### Merge events and log file
Note that the sequence needs to be resorted based on the client timestamp, as logged events could come in out of order. I.e., an errant TCP packet can hit the trigger device after a subsequent packet.

In [ ]:
df = logdf.copy(deep=True).iloc[shift:shift+eventn,:]
df['bytecodeEvent'] = events[:,2]
#(df.bytecode==df.bytecodeEvent).sum()/df.shape[0]
df['ts_event'] = events[:,0]
df.sort_values('client_ts', inplace=True)
df.reset_index(inplace=True, drop=True)
df['delay'] = ((df.trigger_ts - df.client_ts) / eeg_sample_interval_ms).round().astype(int)
df['ts_event_corrected'] = df.ts_event - df.delay

# TODO: estimate network delay from round-trip times and use that to get an accurate client/tirgger clock bias
clock_bias = df.delay.mean()
print('clock_bias = %0.2fms' % clock_bias)

### Synthesize a corrected event sequence
This new sequence takes into account the random delay from one even to the next and the average network delay. Because event trigger packets can arrive out-of-order, they needed to be resorted above to apply the proper delays. But now that everything is corrected, we can resort them based on the event timestamp so mne doesn't complain about a non-chronological event sequence.

In [ ]:
eventdf_new = pd.DataFrame(columns=['ts','diff','code'])
# clock_bias is in milliseconds-- convert to the EEG time stamps
eventdf.ts = df.ts_event_corrected.values + int(round(clock_bias / eeg_sample_interval_ms))
eventdf['diff'] = 0
eventdf.code = 1
eventdf.drop_duplicates(subset='ts', keep='first', inplace=True)
eventdf = eventdf.sort_values('ts').reset_index(drop=True)
#plt.plot(eventdf.ts)

In [ ]:
raw_no_ref, _ = mne.set_eeg_reference(raw.load_data().filter(l_freq=None, h_freq=45), [])
#raw_no_ref, _ = mne.set_eeg_reference(raw.load_data(), [])
reject = dict(eeg=180e-6) # 180e-6, eog=150e-6)
event_id, tmin, tmax = {'visual': 1}, -0.1, 0.5
epochs_params = dict(events=eventdf.values, event_id=event_id, tmin=tmin, tmax=tmax, reject=reject)
evoked_no_ref = mne.Epochs(raw_no_ref, **epochs_params).average()
del raw_no_ref  # save memory

In [ ]:
title = 'EEG Original reference'
evoked_no_ref.plot(titles={'eeg':'title'}, time_unit='ms')#, picks=['O1','O2','P3','P4'])
evoked_no_ref.plot_topomap(times=[0.075,0.1,0.125,.15,.175], size=1.0, title=title, time_unit='s')

### Sample code for doing frequency analysis

In [ ]:
occ = raw.get_data(['O1','O2'])

In [ ]:
ft = np.fft.rfft(occ)
T = eeg_sample_interval_ms / 1000
xf = np.linspace(0.0, 1.0/(2.0*T), int(np.ceil(occ.shape[1]/2))+1)

In [ ]:
fig = plt.Figure(figsize=(12,6))
plt.plot(xf[100:15000], np.abs(ft[1,100:15000]))

In [ ]:
logdf.head()

In [ ]:
plt.plot(df.client_ts)

In [ ]:
df.head()

In [ ]:
raw.get_data().shape